# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [81]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [98]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Max Temp",
    scale = 1,
    color = "Country"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [83]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[city_data_df["Max Temp"]>=20]
ideal_city_df = ideal_city_df[ideal_city_df["Max Temp"]<=30]
ideal_city_df = ideal_city_df[ideal_city_df["Humidity"]<=60]
ideal_city_df = ideal_city_df[ideal_city_df["Cloudiness"]<=50]
ideal_city_df = ideal_city_df[ideal_city_df["Wind Speed"]<=6]


# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
218,218,sao teotonio,37.5128,-8.7071,22.45,60,31,4.98,PT,1666108367
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417
317,317,salalah,17.0151,54.0924,25.49,59,7,1.67,OM,1666108424


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
152,kahului,US,20.8947,-156.4700,60,
218,sao teotonio,PT,37.5128,-8.7071,60,
229,san cristobal,VE,7.7669,-72.2250,45,
240,tikaitnagar,IN,26.9500,81.5833,59,
296,jalu,LY,29.0331,21.5482,37,
305,kingman,US,35.1894,-114.0530,49,
317,salalah,OM,17.0151,54.0924,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [87]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {
    "limit":20,
    "apiKey": geoapify_key,
    "categories" : category 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filters"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address =  response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(longitude)
        print(latitude)
        print(response)

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: فندق إنتركونتيننتال
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: فندق القصر
kahului - nearest hotel: Maui Seaside Hotel
sao teotonio - nearest hotel: Orada Velha
san cristobal - nearest hotel: Terra Sur
tikaitnagar - nearest hotel: Maa patesawri sweet hous
jalu - nearest hotel: فندق جالو السياحي
kingman - nearest hotel: Home2 Suites by Hilton Kingman
salalah - nearest hotel: Muscat International Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: Shivpuri Hotel
saint-francois - nearest hotel: Chez Lily
kalemie - nearest hotel: Hotel du Lac
lakki marwat - nearest hotel: Madina Hotel
tikrit - nearest hotel: فندق بلازا
lahij - nearest hotel: عمارة الجهوري
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,فندق إنتركونتيننتال
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,فندق القصر
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
218,sao teotonio,PT,37.5128,-8.7071,60,Orada Velha
229,san cristobal,VE,7.7669,-72.2250,45,Terra Sur
240,tikaitnagar,IN,26.9500,81.5833,59,Maa patesawri sweet hous
296,jalu,LY,29.0331,21.5482,37,فندق جالو السياحي
305,kingman,US,35.1894,-114.0530,49,Home2 Suites by Hilton Kingman
317,salalah,OM,17.0151,54.0924,59,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "Country",
    hover_cols = ["City", "Hotel Name"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City,Hotel Name)